In [10]:
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer, util

In [ ]:

class CSVEmbeddingSearcher:
    def __init__(
        self,
        csv_path: str,
        embedding_column: str,
        model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    ):
        """
        :param csv_path: Путь к CSV-файлу.
        :param embedding_column: Название столбца, значение которого будем преобразовывать в вектор.
        :param model_name: Название модели Sentence Transformers.
        """
        self.csv_path = csv_path
        self.embedding_column = embedding_column
        self.model_name = model_name
        
        df = pd.read_csv(self.csv_path, sep=";", header=0)
        
        # 2) Оставляем только первые 5 столбцов (если их больше)
        if df.shape[1] > 5:
            df = df.iloc[:, :5]
        

        # 3) Удаляем строки, у которых в первой колонке (df.columns[0]) отсутствует значение (NaN)
        df.dropna(subset=[df.columns[0]], inplace=True)
        
        # (Опционально, если нужно также убрать случаи, когда 
        # значение в первой колонке — пустая строка)
        df = df[df[df.columns[0]].astype(str).str.strip() != ""]
        self.df = df
        
        
        # Загрузим модель (один раз)
        start_load = time.time()
        self.model = SentenceTransformer(self.model_name)
        self.model_load_time = time.time() - start_load
        
        # Создадим 6-й столбец с эмбеддингами
        self._create_embedding_column()
    
    def _create_embedding_column(self):
        """
        Приватный метод: проходится по датасету,
        кодирует значение из embedding_column в вектор
        и кладёт в новый столбец 'embedding_vector'.
        """
        if self.embedding_column not in self.df.columns:
            raise ValueError(f"Столбец '{self.embedding_column}' не найден в первых 5 колонках CSV.")
        
        embeddings_list = []
        start_time = time.time()
        
        # Кодируем каждую строку — для больших объёмов лучше использовать batch_size, но здесь для наглядности всё итеративно.
        for _, row in self.df.iterrows():
            text_value = str(row[self.embedding_column])  # на случай, если там не строка
            emb = self.model.encode(text_value, convert_to_numpy=True)
            embeddings_list.append(emb)
        
        self.df["embedding_vector"] = embeddings_list
        self.embedding_build_time = time.time() - start_time
    
    def search_nearest(self, query: str, top_k: int = 1):
        """
        Ищет в датафрейме ближайшие по косинусному сходству векторы (колонка 'embedding_vector')
        для заданного query. Возвращает top_k совпадений с их метаданными (индекс, похожесть).
        
        :param query: Текст для поиска ближайшего совпадения.
        :param top_k: Сколько ближайших совпадений вернуть.
        :return: Список словарей вида:
            [
              {
                "index": индекс_в_исходном_df,
                "similarity": сходство,
                "row_data": <запись датафрейма целиком>
              },
              ...
            ]
        """
        # Генерируем вектор для запроса
        query_emb = self.model.encode(query, convert_to_tensor=True)
        
        # Собираем все эмбеддинги из DataFrame в один массив
        all_emb = np.vstack(self.df["embedding_vector"].values)
        
        # Вычисляем косинусное сходство разом
        similarities = util.pytorch_cos_sim(query_emb, all_emb)[0].cpu().numpy()  # shape: [num_rows]
        
        # Отсортируем индексы по убыванию сходства
        sorted_indices = np.argsort(-similarities)
        
        # Возьмём top_k результатов
        top_indices = sorted_indices[:top_k]
        
        results = []
        for idx in top_indices:
            sim_score = float(similarities[idx])
            row_data = self.df.iloc[idx].to_dict()
            
            results.append({
                "index": idx,
                "similarity": sim_score,
                "row_data": row_data
            })
        
        return results
    
    def get_stats(self):
        """
        Возвращает простую статистику:
         - время загрузки модели,
         - время на создание эмбеддингов,
         - текущее количество строк и столбцов в датафрейме.
        """
        return {
            "model_name": self.model_name,
            "model_load_time": self.model_load_time,
            "embedding_build_time": getattr(self, "embedding_build_time", None),
            "data_shape": self.df.shape
        }



In [ ]:
csv_path = "ОКС.csv"            
embed_col = "Наименование"      

# Создаём объект (модель загрузится, эмбеддинги построятся)
searcher = CSVEmbeddingSearcher(csv_path, embed_col)

In [18]:
searcher.df.head()

,Подкласс 3,Наименование,Определение,Источник,УИН,embedding_vector
3,AAAA,Здание (сооружение) конюшни,Подразделение сельскохозяйственного предприяти...,Приказ Минстроя России от 02.11.2022 N 928/пр...,KSI030000005,"[-0.08677673, 0.18554646, -0.016069654, 0.0528..."
4,AAAB,Здание (сооружение) содержания молодняка конев...,NaN,Приказ Минстроя России от 02.11.2022 N 928/пр,KSI030001046,"[-0.08405774, 0.2270738, -0.01104456, 0.083461..."
6,AABA,Здание содержания пушных зверей,Подразделение сельскохозяйственного предприяти...,Приказ Минстроя России от 02.11.2022 N 928/пр...,KSI030000009,"[-0.028154934, 0.16233127, -0.017938629, 0.005..."
7,AABB,Здание цеха производства пушнины,NaN,Приказ Минстроя России от 02.11.2022 N 928/пр,KSI030001047,"[-0.078826144, 0.05641119, -0.018503794, 0.061..."
8,AABC,Сооружение шеда,"Навес с двухскатной крышей, под которым находя...",Приказ Минстроя России от 02.11.2022 N 928/пр ...,KSI030001048,"[-0.072845295, 0.032042157, -0.020272423, 0.06..."


In [17]:

# Статистика
# stats = searcher.get_stats()
# print("СТАТИСТИКА:", stats)

# Пример поиска
query_text = "Склад радиактивных"
top_k = 3
results = searcher.search_nearest(query_text, top_k=top_k)

print(f"\nТоп-{top_k} результатов поиска для запроса: '{query_text}'")
for res in results:
    # print(f"- Индекс: {res['index']}, Сходство: {res['similarity']:.3f}, Строка: {res['row_data']}")
    print(f"- Индекс: {res['index']}, Сходство: {res['similarity']:.3f}, Строка: {res['row_data']}")


Топ-3 результатов поиска для запроса: 'Склад радиактивных'
- Индекс: 857, Сходство: 0.880, Строка: {'Подкласс 3': 'SAAP', 'Наименование': 'Склад (хранилище) радиоактивных веществ и материалов\n', 'Определение': 'Пункты хранения ядерных материалов и радиоактивных веществ, пункты хранения, хранилища радиоактивных отходов (далее - пункты хранения) - стационарные объекты и сооружения, не относящиеся к ядерным установкам, радиационным источникам и предназначенные для хранения ядерных материалов и радиоактивных веществ, хранения или захоронения радиоактивных отходов', 'Источник': 'Федеральный закон от 21.11.1995 N 170-ФЗ', 'УИН': 'KSI030001786', 'embedding_vector': array([-5.88752218e-02,  1.30610362e-01, -1.27337947e-02,  4.27992009e-02,
        1.11402042e-01,  5.41903265e-02,  7.55309127e-03,  9.21991467e-02,
        6.86277747e-02,  3.43605243e-02, -1.18940361e-01,  8.83256719e-02,
       -1.72101676e-01,  1.21862562e-02,  3.68472077e-02,  3.69631685e-02,
        8.15299898e-02, -6.5886